In [16]:
import torch
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig

wandb_logger = WandbLogger(project="RRHF")

In [3]:
# Set the number of categories
num_categories = 5

# Load the base model configuration
config = AutoConfig.from_pretrained("cjvt/sloberta-si-nli")
config.num_labels = num_categories

tokenizer = AutoTokenizer.from_pretrained("cjvt/sloberta-si-nli")
model = AutoModelForSequenceClassification.from_pretrained("cjvt/sloberta-si-nli", config=config, ignore_mismatched_sizes=True)

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at cjvt/sloberta-si-nli and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([5, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
data = pd.read_csv('/kaggle/input/oasst1-sl/context_reply_pairs.csv', sep=';')

In [18]:
data.head()

,prompt,reply,rank,split
0,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Kaj pa igranje lažnivca? Za to igro ...,0,train
1,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Uno: Je klasična igra za otroke in o...,1,train
2,UPORABNIK: V mojo hišo bodo prispeli 3 prijate...,ASISTENT: Igrate lahko:\r\n - 8 noro\r\n - cel...,2,train
3,UPORABNIK: Napišite hvalnico šolskemu taboru v...,ASISTENT: Spomnim se vas prijatelji\r\nIn spet...,0,train
4,UPORABNIK: Ali kaj bolj veselega?,ASISTENT: ♪ Spakirajte in se pridružite zabavi...,0,train


In [19]:
data.loc[data['rank'] > 4, 'rank'] = 4
data['combined'] = "UPORABNIK: " + data['prompt'] + " ASISTENT: " + data['reply']

In [20]:
train_data = data[data.split == 'train']
val_data   = data[data.split == 'val']

train_texts = train_data['combined'].tolist()
train_labels = train_data['rank'].tolist()

val_texts = val_data['combined'].tolist()
val_labels = val_data['rank'].tolist()

In [21]:
# Tokenize the text data
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

# Convert labels to numerical format
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)

In [22]:
# Create a custom dataset class
class ReplyClassificationDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item
    
    def __len__(self):
        return len(self.labels)

train_dataset = ReplyClassificationDataset(train_encodings, train_labels)
val_dataset = ReplyClassificationDataset(val_encodings, val_labels)

In [10]:
class ReplyClassificationModel(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, **inputs):
        return self.model(**inputs)

    def training_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = self.forward(input_ids=input_ids, attention_mask=attention_mask)
        loss = torch.nn.functional.cross_entropy(outputs.logits, labels)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        labels = batch['labels']

        outputs = self.forward(input_ids=input_ids, attention_mask=attention_mask)
        loss = torch.nn.functional.cross_entropy(outputs.logits, labels)

        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)



    def configure_optimizers(self):
        return torch.optim.AdamW(self.parameters(), lr=1e-5)

In [11]:
# Initialize the PyTorch Lightning model
pl_model = ReplyClassificationModel(model)

checkpoint = pl.callbacks.ModelCheckpoint(
    dirpath="/kaggle/working",
    filename='best-model',
    save_top_k=2,
    verbose=True,
    monitor="val_loss",
    mode="min"
)

# Set up the PyTorch Lightning trainer
trainer = pl.Trainer(accelerator="auto", 
                     devices=2, 
                     max_epochs=2,
                     callbacks = checkpoint,
                     logger=wandb_logger)

In [ ]:
# Train the model
trainer.fit(pl_model, DataLoader(train_dataset, batch_size=8, shuffle=True), DataLoader(val_dataset, batch_size=16))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /kaggle/working exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('train_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(


In [23]:
k = 422
train_data.iloc[k].combined

'UPORABNIK: UPORABNIK: Mi lahko poveste, katera odločitev vrhovnega sodišča je dala ljudem v Združenih državah individualno pravico do nošenja strelnega orožja? Ali lahko povzamete primer in tudi odločitev? ASISTENT: ASISTENT: Odločitev vrhovnega sodišča, ki je ljudem v Združenih državah dala individualno pravico do nošenja strelnega orožja, je bila District of Columbia proti Hellerju (2008). Primer je vložil Dick Heller, prebivalec okrožja Columbia, ki je izpodbijal prepoved posedovanja pištol v okrožju. Vrhovno sodišče je razsodilo, da drugi amandma ščiti posameznikovo pravico do posedovanja strelnega orožja, ki ni povezano s službo v milici, in da okrožna prepoved posedovanja pištole krši to pravico.'

In [ ]:
model_name = 'sloberta-si-rrhf-context'
pl_model.model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

In [25]:
tokenizer = AutoTokenizer.from_pretrained("vh-student/sloberta-si-rrhf")
model = AutoModelForSequenceClassification.from_pretrained("vh-student/sloberta-si-rrhf")

# Define your custom reply
custom_reply = '''UPORABNIK: Kako se imenujejo očetje domovine Dominikanske republike? ASISTENT: V Dominikanski republiki se za očete države štejejo Juan Pablo Duarte, Francisco del Rosario Sánchez in Matías Ramón Mella. Ti trije možje so vodili boj za neodvisnost Dominikanske republike od Haitija in februarja 1844 ustanovili republiko kot neodvisno državo.\n\nJuan Pablo Duarte velja za ustanovitelja Dominikanske republike in je v državi zelo cenjen narodni heroj. Francisco del Rosario Sánchez in Matías Ramón Mella prav tako veljata za pomembna narodna junaka zaradi njune vloge v boju za neodvisnost.\n\nTi trije možje so sestavni del zgodovine in kulture Dominikanske republike ter se jih spominjajo in častijo zaradi njihove žrtve in odločnosti v boju za neodvisnost in svobodo svoje države.'''
# Tokenize the custom reply
inputs = tokenizer.encode_plus(custom_reply, padding="max_length", truncation=True, max_length=128, return_tensors="pt")

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)

# Get the predicted label
predicted_label = torch.argmax(outputs.logits).item()

# Print the predicted label
print("Predicted label:", predicted_label)
probas = torch.nn.functional.softmax(outputs.logits, dim=1)[0]
s  = probas[0]^1 + probas[1]^2 + probas[2]^3 + probas[3]^4 + probas[4]^5

Predicted label: 1
Probabilities: tensor([[0.2470, 0.4846, 0.2554, 0.0101, 0.0028]])
